In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

# Michael change
#from tensorflow import keras
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.5.0


Using TensorFlow backend.


# Get data


In [36]:
data_stack = pd.read_csv('DB/stack-overflow-data.csv')
read = open('DB/stack-overflow-data-tags')
tags_stack = read.readline().split('|')
read.close()

In [37]:
data_stack.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [38]:
data_stack.rename(columns={'post' : 'Body', 'tags': 'Tag'}, inplace=True)
data_stack.head()

,Body,Tag
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [39]:
data_free = pd.read_csv('DB/freelance_data.csv', index_col=0)
read = open('DB/freelance_data_tags')
tags_free = read.readline().split('|')
read.close()

In [40]:
data_free.head()

,Body,Tag
0,I am a freelance web developer. I have built ...,"['contracts', 'changes', 'design']"
1,I recently started freelancing and found mysel...,"['attracting-clients', 'project', 'difficult-c..."
2,"When working through the day, I may work on ma...","['time-management', 'time-tracking']"
3,I have a project where the client initially re...,['project-management']
4,I started a project given by client and it wen...,"['time-management', 'project-management', 'com..."


In [41]:
data_start = pd.read_csv('DB/startup_data.csv', index_col=0)
read = open('DB/startup_data_tags')
tags_start = read.readline().split('|')
read.close()

In [42]:
data_start.head()

,Body,Tag
0,"After registering my small LLC, I signed up fo...","['llc', 'new-hampshire', 'united-states']"
1,"If I have a startup that produces apps, is the...","['tech-company', 'mobile-apps', 'equity']"
2,"In short, for my very first business plan whic...",['business-plan']
3,We know that venture capitalists keep a keen e...,"['intellectual-property', 'investment', 'ventu..."
4,"I'm thinking of doing some freelance work, and...","['united-kingdom', 'freelancing']"


In [43]:
all_tags = tags_stack.copy()
all_tags.extend(tags_free.copy())
all_tags.extend(tags_start.copy())

In [44]:
all_data = pd.DataFrame(columns=['Body', 'Tag'])

In [45]:
all_data = all_data.append(data_free)
all_data = all_data.append(data_start)
all_data = all_data.append(data_stack)

In [48]:
all_data.head()

,Body,Tag
0,I am a freelance web developer. I have built ...,"['contracts', 'changes', 'design']"
1,I recently started freelancing and found mysel...,"['attracting-clients', 'project', 'difficult-c..."
2,"When working through the day, I may work on ma...","['time-management', 'time-tracking']"
3,I have a project where the client initially re...,['project-management']
4,I started a project given by client and it wen...,"['time-management', 'project-management', 'com..."


In [49]:
from sklearn.cross_validation import train_test_split

In [50]:
train_posts, test_posts, train_tags, test_tags = train_test_split(all_data.Body, all_data.Tag, 
                                                                  random_state=42, test_size=0.2)

In [71]:
def process_tags(tags):
    tmp = list(tags)
    for i in range(len(tmp)):
        if tmp[i][0] == '[' and tmp[i][-1] == ']':
            tmp[i] = tmp[i][1:-1].split(',')
            for j in range(len(tmp[i])):
                tmp[i][j] = tmp[i][j].strip()
                tmp[i][j] = tmp[i][j][1:-1]
        else:
            tmp[i] = [tmp[i]]
    return tmp

In [72]:
test_tags = process_tags(test_tags)
test_tags

[['javascript'],
 ['java'],
 ['angularjs'],
 ['c++'],
 ['c++'],
 ['android'],
 ['jquery'],
 ['angularjs'],
 ['jquery'],
 ['.net'],
 ['javascript'],
 ['objective-c'],
 ['php'],
 ['python'],
 ['css'],
 ['objective-c'],
 ['asp.net'],
 ['android'],
 ['objective-c'],
 ['sql'],
 ['mysql'],
 ['ios'],
 ['jquery'],
 ['php'],
 ['mysql'],
 ['java'],
 ['asp.net'],
 ['sql'],
 ['html'],
 ['asp.net'],
 ['c++'],
 ['iphone'],
 ['iphone'],
 ['c++'],
 ['united-states', 'legal', 'business-registration'],
 ['javascript'],
 ['ios'],
 ['objective-c'],
 ['canada'],
 ['c++'],
 ['c++'],
 ['jquery'],
 ['lean', 'competitive-events', 'lean-startup-machine'],
 ['mysql'],
 ['.net'],
 ['ios'],
 ['mysql'],
 ['java'],
 ['angularjs'],
 ['css'],
 ['c#'],
 ['angularjs'],
 ['c'],
 ['java'],
 ['android'],
 ['javascript'],
 ['c'],
 ['android'],
 ['javascript'],
 ['business-model', 'data'],
 ['advertising'],
 ['angularjs'],
 ['equity', 'idea'],
 ['c'],
 ['c#'],
 ['.net'],
 ['ios'],
 ['android'],
 ['php'],
 ['c'],
 ['android']

In [73]:
train_tags = process_tags(train_tags)

In [77]:
test_posts = list(test_posts)
train_posts = list(train_posts)